<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.64
    Uninstalling yfinance-0.2.64:
      Successfully uninstalled yfinance-0.2.64
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-06-30 09:19:44
-------------------
qualified stocks: 93
with latest results: 29
still star stocks: 18
-------------------
Initial Investment:  1.17 C
CY Investment:  1.44 C
Reserve:  6.47 K
Current:  1.35 C
-------------------
Today PnL: 61.99 K (0.46%)
Current PnL: -14.63 L (-10.19%)
CY Booked + Current PnL: -7.89 L (-5.5%)
-------------------
Total profit:  3.79 L
Total loss:  -18.41 L
-------------------
Total Booked + Current PnL: 19.09 L (16.38%)
Total Booked PnL: 33.72 L (28.93%)
Curr Year Booked PnL: 6.74 L (5.0%)
Prev Year Booked PnL: 26.98 L (23.15%)
Est FTT:  2.07 C
Est FTT PnL: 71.95 L (53.33%)
Deployed:  1.17 C
Current:  1.35 C
CAGR/XIRR %: 9.99%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ADANIPORTS,1103.69,1583.00,2.03,M-LC,43.01,102453.0,24091.0,9938.0,0.19,30.74,9.70,43.43,77.0,2.42,0.76,37.70,XY24,NTT,MISC
84,UJJIVANSFB,52.77,53.00,52.86,M-SC,92.47,133056.0,-9423.0,10046.0,1.00,-6.61,7.55,0.44,243.0,-0.94,0.99,57.04,OX40N,NTT,BANKS
36,HINDALCO,651.95,761.55,-19.08,H-LC,86.02,111224.0,6912.0,10622.0,-0.32,6.63,9.55,16.81,27.0,0.65,0.82,23.74,X5K,ATH,METALS
85,UNIONBANK,123.87,163.00,-1.39,M-LC,56.99,172312.0,31472.0,13027.0,1.67,22.35,7.56,31.59,89.0,2.42,1.28,49.68,XY24,NTT,BANKS
68,SBIN,760.30,863.00,-11.51,M-LC,30.11,215048.0,13568.0,13656.0,0.76,6.73,6.35,13.51,76.0,0.99,1.59,20.18,XY25,NTT,BANKS


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,KPIGREEN,497.21,732.16,23.50,H-SC,54.84,126529.0,1232.0,57976.0,1.41,0.98,45.82,47.25,139.0,0.02,0.94,58.30,MH,ATH,POWER
52,LALPATHLAB,2804.19,3545.00,-33.01,H-SC,39.78,154160.0,-70.0,40822.0,1.23,-0.05,26.48,26.42,125.0,-0.00,1.14,21.57,X40N,NTT,HEALTHCARE
15,BLUESTARCO,1646.70,2337.55,-12.17,H-SC,77.42,164760.0,90.0,69001.0,1.45,0.05,41.88,41.95,121.0,0.00,1.22,7.52,X40N,ATH,AC
35,HEROMOTOCO,4311.81,6039.03,-6.19,H-MC,21.51,151210.0,297.0,60151.0,0.99,0.20,39.78,40.06,102.0,0.00,1.12,23.68,AR,ATH,AUTO
57,NATIONALUM,189.63,247.44,-49.40,H-MC,74.19,101310.0,806.0,29836.0,0.17,0.80,29.45,30.49,97.0,0.03,0.75,33.47,MH,ATH,METALS


In [11]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,ROUTE,1558.40,2249.26,-37.23,M-SC,78.49,67399.0,-35455.0,81054.0,2.19,-34.47,120.26,44.33,192.0,-0.44,0.50,11.92,SR,ATH,IT
63,REPCOHOME,515.07,880.00,-51.56,H-SC,94.62,178423.0,-26575.0,171821.0,1.98,-12.96,96.30,70.85,133.0,-0.15,1.32,44.03,XY24,NTT,FINANCE
11,BANDHANBNK,214.25,400.00,-26.39,H-SC,88.17,219402.0,-29128.0,244589.0,1.90,-11.72,111.48,86.70,129.0,-0.12,1.63,45.15,XY24,NTT,BANKS
67,SATIN,199.77,274.00,-19.84,H-SC,55.91,157176.0,-38798.0,111611.0,1.86,-19.80,71.01,37.16,130.0,-0.35,1.17,17.38,XY24,NTT,FINANCE
12,BANKINDIA,116.91,190.00,-30.06,M-MC,44.09,182853.0,3045.0,109364.0,1.82,1.69,59.81,62.52,171.0,0.03,1.36,36.36,XR,NTT,BANKS


In [12]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,RAJESHEXPO,517.65,518.0,1825.51,M-SC,37.63,52528.0,-84649.0,84743.0,-2.50,-61.71,161.33,0.07,231.0,-1.00,0.39,30.64,OX40N,NTT,JEWELLERY
26,DEN,52.79,75.0,112.44,L-SC,32.26,59984.0,-24480.0,60014.0,-1.60,-28.98,100.05,42.07,258.0,-0.41,0.44,27.39,AR,NTT,ENTERTAINMENT
91,WHIRLPOOL,1167.49,2270.0,-38.05,M-SC,89.25,204389.0,32768.0,129296.0,-0.88,19.09,63.26,94.43,198.0,0.25,1.52,50.85,XR,NTT,DURABLES
72,SIS,477.00,477.0,2252.33,M-SC,62.37,63291.0,-17799.0,17797.0,-0.75,-21.95,28.12,0.00,227.0,-1.00,0.47,28.27,OX40N,NTT,MISC
5,APOLLOHOSP,6904.43,8285.0,-19.95,H-LC,69.89,109072.0,5506.0,15205.0,-0.51,5.32,13.94,20.00,30.0,0.36,0.81,20.14,X40N,BTT,HEALTHCARE


In [13]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,UJJIVANSFB,52.77,53.0,52.86,M-SC,92.47,133056.0,-9423.0,10046.0,1.00,-6.61,7.55,0.44,243.0,-0.94,0.99,57.04,OX40N,NTT,BANKS
78,TATAELXSI,7332.28,7332.0,-15.29,X-MC,27.96,88886.0,-13766.0,13760.0,0.06,-13.41,15.48,-0.00,58.0,-1.00,0.66,35.12,OX40N,NTT,IT
89,VIPIND,488.80,489.0,-212.52,H-SC,95.70,81169.0,-14147.0,14188.0,-0.23,-14.84,17.48,0.04,154.0,-1.00,0.60,60.10,OX40N,NTT,MISC
50,KANSAINER,299.63,340.0,-67.65,H-SC,25.81,225180.0,-44487.0,80817.0,0.85,-16.50,35.89,13.47,142.0,-0.55,1.67,13.30,XY24,NTT,PAINTS
76,SYMPHONY,1306.42,1306.0,-33.55,M-SC,3.23,141690.0,-29451.0,29401.0,-0.02,-17.21,20.75,-0.03,189.0,-1.00,1.05,1.84,OX40N,NTT,DURABLES


In [14]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
80,TCS,3794.03,4998.00,-17.42,X-LC,33.33,268796.0,-27138.0,121039.0,0.15,-9.17,45.03,31.73,1.0,-0.22,1.99,7.59,X40,BTT,IT
45,INFY,1461.46,2275.00,-10.27,X-LC,49.46,287725.0,26124.0,119492.0,-0.04,9.99,41.53,55.67,2.0,0.22,2.13,16.70,X40,BTT,IT
37,HINDUNILVR,2413.81,2723.00,-17.36,X-LC,26.88,258854.0,-11493.0,46128.0,0.19,-4.25,17.82,12.81,6.0,-0.25,1.92,8.26,XY25,NTT,FMCG
58,NESTLEIND,2268.60,2755.00,-9.17,X-LC,48.39,288171.0,22745.0,34177.0,0.17,8.57,11.86,21.44,9.0,0.67,2.14,15.30,XY25,NTT,FMCG
16,BRITANNIA,4983.33,6446.05,16.49,X-LC,65.59,266915.0,37682.0,29601.0,0.14,16.44,11.09,29.35,10.0,1.27,1.98,26.83,XY25,ATH,FMCG


In [15]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,UNIONBANK,123.87,163.0,-1.39,M-LC,56.99,172312.0,31472.0,13027.0,1.67,22.35,7.56,31.59,89.0,2.42,1.28,49.68,XY24,NTT,BANKS
3,ADANIPORTS,1103.69,1583.0,2.03,M-LC,43.01,102453.0,24091.0,9938.0,0.19,30.74,9.70,43.43,77.0,2.42,0.76,37.70,XY24,NTT,MISC


In [16]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SAIL,130.64,228.00,89.46,M-MC,45.16,227390.0,2428.0,165222.0,0.27,1.08,72.66,74.53,185.0,0.01,1.69,32.02,XY24,BTT,STEEL
51,KPIGREEN,497.21,732.16,23.50,H-SC,54.84,126529.0,1232.0,57976.0,1.41,0.98,45.82,47.25,139.0,0.02,0.94,58.30,MH,ATH,POWER
19,CAMS,3643.00,5250.99,7.65,H-SC,84.95,121030.0,19026.0,25997.0,0.69,18.65,21.48,44.14,124.0,0.73,0.90,37.92,X40N,ATH,MISC
48,JIOFIN,310.58,387.00,6.70,M-LC,91.40,228620.0,11214.0,42272.0,0.97,5.16,18.49,24.61,95.0,0.27,1.70,62.55,XY24,BTT,FINANCE
3,ADANIPORTS,1103.69,1583.00,2.03,M-LC,43.01,102453.0,24091.0,9938.0,0.19,30.74,9.70,43.43,77.0,2.42,0.76,37.70,XY24,NTT,MISC


In [17]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
91,WHIRLPOOL,1167.49,2270.00,-38.05,M-SC,89.25,204389.0,32768.0,129296.0,-0.88,19.09,63.26,94.43,198.0,0.25,1.52,50.85,XR,NTT,DURABLES
92,WIPRO,243.46,420.00,-4.22,M-LC,73.12,164052.0,13107.0,96348.0,-0.17,8.68,58.73,72.51,69.0,0.14,1.22,13.00,XR,NTT,IT
42,INDIAMART,2327.09,4871.06,-46.77,H-SC,90.32,138568.0,15232.0,119598.0,0.24,12.35,86.31,109.32,122.0,0.13,1.03,36.90,AR,ATH,MISC
57,NATIONALUM,189.63,247.44,-49.40,H-MC,74.19,101310.0,806.0,29836.0,0.17,0.80,29.45,30.49,97.0,0.03,0.75,33.47,MH,ATH,METALS
12,BANKINDIA,116.91,190.00,-30.06,M-MC,44.09,182853.0,3045.0,109364.0,1.82,1.69,59.81,62.52,171.0,0.03,1.36,36.36,XR,NTT,BANKS


In [18]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
23,COFFEEDAY,59.14,80.00,-51.71,L-SC,96.77,71731.0,-41818.0,81867.0,1.58,-36.83,114.13,35.27,261.0,-0.51,0.53,75.56,XR,NTT,HOTELS
55,MASFIN,326.60,402.39,-18.25,H-SC,64.52,93000.0,-4980.0,27714.0,-0.43,-5.08,29.80,23.21,137.0,-0.18,0.69,34.88,XR,ATH,FINANCE
4,ALKYLAMINE,2347.98,4546.37,5.41,X-SC,98.92,98130.0,-2833.0,97365.0,0.89,-2.81,99.22,93.63,64.0,-0.03,0.73,46.16,SR,ATH,CHEMICALS
3,ADANIPORTS,1103.69,1583.00,2.03,M-LC,43.01,102453.0,24091.0,9938.0,0.19,30.74,9.70,43.43,77.0,2.42,0.76,37.70,XY24,NTT,MISC
5,APOLLOHOSP,6904.43,8285.00,-19.95,H-LC,69.89,109072.0,5506.0,15205.0,-0.51,5.32,13.94,20.00,30.0,0.36,0.81,20.14,X40N,BTT,HEALTHCARE


In [19]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
69,SFL,1016.35,1287.00,44.93,H-SC,100.00,191281.0,-70937.0,140764.0,0.61,-27.05,73.59,26.63,144.0,-0.50,1.42,18.93,XY24,NTT,MISC
4,ALKYLAMINE,2347.98,4546.37,5.41,X-SC,98.92,98130.0,-2833.0,97365.0,0.89,-2.81,99.22,93.63,64.0,-0.03,0.73,46.16,SR,ATH,CHEMICALS
49,JPPOWER,18.73,26.20,-31.42,L-SC,97.85,138132.0,-3710.0,60281.0,0.77,-2.62,43.64,39.88,259.0,-0.06,1.02,39.98,XY24,NTT,POWER
23,COFFEEDAY,59.14,80.00,-51.71,L-SC,96.77,71731.0,-41818.0,81867.0,1.58,-36.83,114.13,35.27,261.0,-0.51,0.53,75.56,XR,NTT,HOTELS
89,VIPIND,488.80,489.00,-212.52,H-SC,95.70,81169.0,-14147.0,14188.0,-0.23,-14.84,17.48,0.04,154.0,-1.00,0.60,60.10,OX40N,NTT,MISC


In [20]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.19
1,25,42.47
2,50,71.15


In [21]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.91
LC    34.48
MC    21.62
Name: CurrAlloc%, dtype: float64

In [22]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     31.36
X40      12.34
XY25     12.19
X40N     11.88
XR       10.30
OX40N     8.25
AR        7.87
MH        1.69
X5K       1.49
X200      1.41
SR        1.23
Name: CurrAlloc%, dtype: float64

In [23]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    20.86
M-SC    16.18
H-LC    12.64
X-LC    12.35
H-MC     8.63
M-LC     8.40
X-MC     7.62
M-MC     5.00
X-SC     4.40
L-SC     2.47
L-LC     1.09
L-MC     0.37
Name: CurrAlloc%, dtype: float64

In [24]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.33,-1.22,35.05
IT,11.88,-10.71,65.69
BANKS,8.04,-5.31,48.28
FINANCE,8.02,-11.57,51.96
MISC,6.65,-20.60,67.75
PAINTS,5.69,-13.87,36.31
ELECTRICAL,5.06,-3.09,47.74
HEALTHCARE,4.52,-5.63,36.49
AUTO,4.39,-14.12,64.31


In [25]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2704192.0,25
XR,1071558.0,13
AR,1008570.0,9
X40,655397.0,9
X40N,560445.0,11
XY25,474657.0,8
OX40N,318130.0,11
SR,178419.0,2
MH,87812.0,2


In [26]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,1971222.0,20
M-SC,1760888.0,21
H-LC,565119.0,11
X-LC,509529.0,7
M-MC,494245.0,4
H-MC,472332.0,7
X-MC,424456.0,6
X-SC,325541.0,5
M-LC,312838.0,6


In [27]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,,sum,count
Conviction,Criteria,,
H-SC,XY24,854000.0,6
M-SC,XY24,814154.0,7
H-SC,AR,518308.0,3
M-MC,XY24,384881.0,3
M-SC,XR,332455.0,4
X-LC,X40,240531.0,2
H-SC,X40N,231960.0,4
H-MC,XY24,224039.0,2
H-SC,XR,214692.0,3


In [28]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 28.0 seconds
